# Aula 05 &mdash; Data Scraping com BeautifulSoup

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

In [2]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re

Nessa aula iremos extrair informações sobre os preços de um iPhone 7 usando o site de pesquisa de preços BuscaPé.
Para evitar que várias requisições sejam feitas ao site enquanto testamos nosso código, a página referida pela URL original foi baixada com o auxílio da ferramenta *cURL*.

Uma observação importante ao se trabalhar com dados da internet é prestar atenção na codificação do texto usada na página. O parser pode não conseguir processar a entrada caso a codificação não tenha sido definida corretamente. No exemplo a seguir, inicialmente não colocamos o tipo de codificação usada. Isso acarreta no erro de processamento, pois o parser entende que a página foi codificada com UTF-8, enquanto na verdade foi usado ISO-8859-1 (Latin1). Esse formato é antigo e a ISO não oferece mais suporte a ele. No entanto, ele é um formato amplamente usado no Windows, logo, muitas páginas ainda vão usá-lo.

### Forma em que ocorrerá um erro de codificação

In [6]:
#url = "http://www.buscape.com.br/celular-e-smartphone/iphone/7"
#page = rq.get(url)
#print(page.text)
#parsed_page = BeautifulSoup(page.text,"html.parser")

url = "../2017.1/data/scraping/buscape/7"
parsed_page = BeautifulSoup(open(url),"html.parser")
#print(parsed_page)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 138816: invalid continuation byte

Observe na descrição do erro acima que o parser assumiu que a codificação do texto era UTF-8.
No entanto, ao verificar o código da página, vemos que ela foi escrita realmente com Latin1.

### Forma em que não ocorrerá um erro de codificação

In [7]:
parsed_page = BeautifulSoup(open(url,encoding="latin1"),"html.parser")
#print(parsed_page)
print(parsed_page.prettify()[:1000])

<!DOCTYPE html>
<!--[if IE 7]><html lang="en" class="no-js ie7" /><![endif]-->
<!--[if IE 8]><html lang="en" class="no-js ie8" /><![endif]-->
<!--[if IE 9]><html lang="en" class="no-js ie9" /><![endif]-->
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <meta content="68654997CD04154E3039F7ED51DC37CE" name="msvalidate.01">
    <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
     <meta content="buscape" id="author" name="author">
      <meta content="1 days" name="revisit">
       <meta content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, width=device-width" name="viewport">
        <meta content="app-id=br.com.buscape.MainPack" name="google-play-app">
         <meta content="True" name="HandheldFriendly">
          <meta content="320" name="MobileOptimized">
           <meta content="https://www.googleapis.com/auth/userinfo.email" name="google-signin-scope">
            <meta content="760344128140

Ao imprimir o objeto BeautifulSoup vemos apenas o código HTML da página como uma string longa. BeautifulSoup, no entanto, permite que os elementos/tags da página sejam buscada e processadas. Por exemplo, podemos acessar o título da página através da tag *title*.

In [9]:
parsed_page.title

<title>Celular e Smartphone 7 iPhone - Preços no Buscapé</title>

Se quisermos somento o texto do título da página sem as tags, devemos acessar o atributo *text* do título.

In [12]:
parsed_page.title.text

'Celular e Smartphone 7 iPhone - Preços no Buscapé'

Podemos ainda buscar por uma ou todas ocorrências de uma certa tag.

In [34]:
print(parsed_page.find('a'))
parsed_page.find_all('a')[:10]


<a class="brand__thumb" data-gaaction="logo" data-gacategory="topo" data-gaevent="event" href="/" itemprop="url" title="Buscapé"><img alt="Buscapé" itemprop="logo" src="http://image.buscape.com/material/buscape.png" srcset="http://image.buscape.com/material/logo-buscape.svg"/></a>


[<a class="brand__thumb" data-gaaction="logo" data-gacategory="topo" data-gaevent="event" href="/" itemprop="url" title="Buscapé"><img alt="Buscapé" itemprop="logo" src="http://image.buscape.com/material/buscape.png" srcset="http://image.buscape.com/material/logo-buscape.svg"/></a>,
 <a class="js-nav--link nav--link js-navToggleMobile" href="#">Navegue por categorias <i class="nav--ico nav--ico-title ico--arrow ico--arrow-down"></i></a>,
 <a class="js-subnav--link nav--link" data-gaaction="celulares" data-gacategory="topo_menu_categorias" data-gaevent="event" data-galabel="celulares" href="/celular-e-smartphone" itemprop="url" log-cat-attribute="log-cat-attribute" log_id="77"><i class="nav--ico nav--ico-left nav--ico-large ico--celular"></i>Celulares<i class="nav--ico nav--ico-right ico--arrow fl-right"></i></a>,
 <a class="nav--link nav--link-sub" href="/celular-e-smartphone/iphone" itemprop="url"><i class="nav--ico nav--ico-left nav--ico-large"></i>Iphones </a>,
 <a class="nav--link 

Em muitos casos estamos interessados em encontrar somente os links da página sem nos importar com outros elementos da tag ou ainda seu texto. A função *find* retorna um objeto do tipo *Tag* através do qual podemos acessar os diversos atributos da tag. Podemos ainda listar quais atributos estão presentes na tag como no exemplo a seguir:

In [35]:
# recuperando o primeiro link da pagina
first_link = parsed_page.find('a')

# acessando a propriedade href (que contem efetivamente o link)
print(first_link.get('href'))
print(repr(first_link.attrs))

print(first_link['class'])

/
{'data-gacategory': 'topo', 'class': ['brand__thumb'], 'data-gaevent': 'event', 'href': '/', 'data-gaaction': 'logo', 'title': 'Buscapé', 'itemprop': 'url'}
['brand__thumb']


Essas funções permitem encontrar facilmente informações úteis nas páginas. Por exemplo, neste exemplo em particular, estamos interessados em saber os preços dos diversos modelos de iPhone disponíveis nas lojas pesquisadas pelo BuscaPé.
Se inspecionarmos o código da página, percebemos que as informações estão localizadas dentro da tag ***div*** (que para nós não importa o significado) cujo atributo ***class*** seja igual a ***'bui-card bui-product'***. Assim podemos buscar exatamente por esses elementos usando a função `find_all`.

In [8]:
#precoToFloat = lambda x: float(re.sub(',','.',re.sub('R\$\s+|\.','',x)))
precoToFloat = lambda x: float(re.sub(',','.',re.sub('[a-zA-Z\$\s\.]+','',x)))
listCelulares = []
for a in parsed_page.find_all('li',itemtype=re.compile('Product')):
#    print(a.find('div',class_='bui-product__name').text)
    modelo, capacidade = re.match('(iPhone \dS?(?:\sPlus)?)\s*(\d+)GB',\
                                  re.sub('Smartphone Apple ','',\
                                         a.find('div',class_='bui-product__name').text)).groups()
#    print("Modelo: ",modelo, "Capacidade: ",capacidade)
#    print(precoToFloat(a.find(itemprop='lowPrice').text))
    menorPreco = precoToFloat(a.find(itemprop='lowPrice').text)
    maiorPreco = precoToFloat(a.find(itemprop='highPrice').text)
    disponibilidade = int(a.find('span',attrs={'name': 'offerCount'}).text)
    listCelulares.append(dict(zip(['modelo','capacidade','menorPreco','maiorPreco','disponibilidade'],\
                                  [modelo,int(capacidade),menorPreco,maiorPreco,disponibilidade])))
listCelulares

[{'capacidade': 16,
  'disponibilidade': 15,
  'maiorPreco': 2559.99,
  'menorPreco': 1381.25,
  'modelo': 'iPhone 5S'},
 {'capacidade': 128,
  'disponibilidade': 14,
  'maiorPreco': 3959.12,
  'menorPreco': 3399.0,
  'modelo': 'iPhone 7'},
 {'capacidade': 32,
  'disponibilidade': 14,
  'maiorPreco': 3607.12,
  'menorPreco': 2931.65,
  'modelo': 'iPhone 7'},
 {'capacidade': 128,
  'disponibilidade': 13,
  'maiorPreco': 4499.0,
  'menorPreco': 3849.0,
  'modelo': 'iPhone 7 Plus'},
 {'capacidade': 256,
  'disponibilidade': 13,
  'maiorPreco': 4899.0,
  'menorPreco': 4199.0,
  'modelo': 'iPhone 7 Plus'},
 {'capacidade': 128,
  'disponibilidade': 12,
  'maiorPreco': 4599.0,
  'menorPreco': 2991.12,
  'modelo': 'iPhone 6S'},
 {'capacidade': 32,
  'disponibilidade': 12,
  'maiorPreco': 4099.0,
  'menorPreco': 3499.0,
  'modelo': 'iPhone 7 Plus'},
 {'capacidade': 256,
  'disponibilidade': 12,
  'maiorPreco': 4899.0,
  'menorPreco': 3497.75,
  'modelo': 'iPhone 7'},
 {'capacidade': 16,
  'disp

Veja que a busca retornada pelo site não é bastante eficaz, uma vez que retornou outros modelos de iPhone. Por se tratar de um exemplo didático, vamos supor que o site tenha retornado modelos que tenham sido adquiridos por usuários que buscavam pelo iPhone 7. Portanto, vamos trabalhar com esses dados sem filtrar os outros modelos.

### Construindo um data frame em pandas usando os dados coletados

O resultado do processamento da página na seção anterior é uma lista de dicionários cada um com as informações referentes a um modelo específico. Para facilitar a manipulação dos dados, podemos criar um data frame com o resultado. A partir daí podemos obter estatísticas sobre a variação de preços, disponibilidade de lojas etc.
O exemplo a seguir mostra como criar o data frame e obter as estatísticas básicas sobre os celulares.


In [28]:
dataCelulares = pd.DataFrame(listCelulares,\
                             columns=['modelo','capacidade','menorPreco','maiorPreco','disponibilidade'])
print(dataCelulares.describe())
dataCelulares

       capacidade   menorPreco   maiorPreco  disponibilidade
count   15.000000    15.000000    15.000000        15.000000
mean    88.533333  2990.282000  3934.018667         9.866667
std     81.104753   779.716991   920.101374         4.596065
min     16.000000  1381.250000  2149.000000         1.000000
25%     32.000000  2385.360000  3323.085000         7.000000
50%     64.000000  2991.120000  3999.000000        12.000000
75%    128.000000  3499.000000  4599.000000        13.000000
max    256.000000  4199.000000  5305.000000        15.000000


,modelo,capacidade,menorPreco,maiorPreco,disponibilidade
0,iPhone 5S,16,1381.25,2559.99,15
1,iPhone 7,128,3399.00,3959.12,14
2,iPhone 7,32,2931.65,3607.12,14
3,iPhone 7 Plus,128,3849.00,4499.00,13
4,iPhone 7 Plus,256,4199.00,4899.00,13
5,iPhone 6S,128,2991.12,4599.00,12
6,iPhone 7 Plus,32,3499.00,4099.00,12
7,iPhone 7,256,3497.75,4899.00,12
8,iPhone 6,16,2249.10,4599.00,11
9,iPhone 6S,16,2464.41,3999.00,11


Observando os dados vemos que existem cotações para um mesmo modelo com diferentes capacidades. Nesse caso, podemos verificar o preço médio de cada modelo independentemente da capacidade para avaliar se existem grandes diferenças de preço. Como vimos anteriormente, isso é feito usando o processo de *split-apply-combine* (função **groupby** em pandas).

In [29]:
dataCel_modelo = dataCelulares.groupby('modelo')
dataCel_modelo.agg({'capacidade':lambda x: ', '.join(map(str,sorted(x))),\
                    'menorPreco':np.min, \
                    'maiorPreco':np.max})

,capacidade,maiorPreco,menorPreco
modelo,,,
iPhone 5S,"16, 32",2559.99,1381.25
iPhone 6,"16, 64, 128",4599.00,2249.10
iPhone 6S,"16, 32, 64, 128",5305.00,2464.41
iPhone 7,"32, 128, 256",4899.00,2931.65
iPhone 7 Plus,"32, 128, 256",4899.00,3499.00


A tabela acima nos permite fazer uma análise mais criteriosa dos celulares disponíveis e custo/benefício de cada modelo. Podemos, por exemplo, optar pela escolha mais barata e avaliar as lojas que vendem o modelo 5S. Percebemos também que por R$200,00 a mais podemos comprar o modelo 6S ao invés do 6 com a mesma capacidade. Como o modelo 6S é equipado com um processador mais rápido, melhor camera, entre outros itens, vemos que se trata de um melhor custo/benefício. (Essa não é uma avaliação técnica! Estou apenas exemplificando o tipo de análise que pode ser feita com os dados)

### Complementando as informações com os dados das lojas

O BuscaPé resume a informação sobre o modelo na página de consulta. Ele exibe as informações básicas do modelo e mostra a variação de preços observadas nas lojas em que o aparelho encontra-se disponível. O usuário deve clicar no link retornado para explorar as lojas que vendem o produto. Vamos agora usar os links encontrados na página inicial para montar uma tabela mais acurada com os valores observados por loja e o link direto para o vendedor. Em seguida, iremos salvar as informações em arquivo.

#### Obtendo os links com os detalhes de cada modelo

Para essa primeira parte, voltaremos ao processamento feito para coletar as informações dos modelos disponíveis. No entanto, dessa vez iremos filtrar apenas as informações do modelo e o link; a informação sobre os preços será obtida para cada loja.

A informação do link é mantida na tag *a*, especificamente no atributo *href*. Assim, devemos filtrar obter essa propriedade. Inspecionando novamente o código da página, vemos que o link para o detalhamento do produto está na primeira tag. Logo, essa será a informação que vamos buscar.

In [10]:
tmp = []
for a in parsed_page.find_all('li',itemtype=re.compile('Product')):
    modelo, capacidade = re.match('(iPhone \dS?(?:\sPlus)?)\s*(\d+)GB',\
                                  re.sub('Smartphone Apple ','',\
                                         a.find('div',class_='bui-product__name').text)).groups()
    link = a.find('a').get('href')    
    print(modelo, capacidade, link)
    tmp.append(((modelo,int(capacidade) ),link))

celulares = dict(tmp)
del tmp

iPhone 5S 16 http://www.buscape.com.br/smartphone-apple-iphone-5s-16gb
iPhone 7 128 http://www.buscape.com.br/smartphone-apple-iphone-7-128gb
iPhone 7 32 http://www.buscape.com.br/smartphone-apple-iphone-7-32gb
iPhone 7 Plus 128 http://www.buscape.com.br/smartphone-apple-iphone-7-plus-128gb
iPhone 7 Plus 256 http://www.buscape.com.br/smartphone-apple-iphone-7-plus-256gb
iPhone 6S 128 http://www.buscape.com.br/smartphone-apple-iphone-6s-128gb
iPhone 7 Plus 32 http://www.buscape.com.br/smartphone-apple-iphone-7-plus-32gb
iPhone 7 256 http://www.buscape.com.br/smartphone-apple-iphone-7-256gb
iPhone 6 16 http://www.buscape.com.br/smartphone-apple-iphone-6-16gb
iPhone 6S 16 http://www.buscape.com.br/smartphone-apple-iphone-6s-16gb
iPhone 6S 32 http://www.buscape.com.br/smartphone-apple-iphone-6s-32gb
iPhone 6S 64 http://www.buscape.com.br/smartphone-apple-iphone-6s-64gb
iPhone 6 64 http://www.buscape.com.br/smartphone-apple-iphone-6-64gb
iPhone 5S 32 http://www.buscape.com.br/smartphone-app

#### Fazendo download das páginas para diretório local

Para evitar de fazer múltiplos acessos ao servidor do BuscaPé, iremos processar os links e salvar as páginas correspondentes numa pasta local.

In [242]:
# essa parte faz o download da pagina referencia para o processamento 
# (mais uma vez para evitar multiplas chamadas ao site enquanto testamos o codigo)
# O simbolo ! serve para executar um comando no shell. 
# Nesse caso, mudamos o diretorio, salvamos a pagina usando curl
# e voltamos para o diretorio de origem
for link in celulares.values():
    !cd 2017.1/data/scraping/buscape/ && curl -O {link} && cd -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  325k    0  325k    0     0   270k      0 --:--:--  0:00:01 --:--:--  270k
/Users/rvimieiro/Documents/ufpe/aulas/data_science
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  320k    0  320k    0     0   360k      0 --:--:-- --:--:-- --:--:--  360k
/Users/rvimieiro/Documents/ufpe/aulas/data_science
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  323k    0  323k    0     0   545k      0 --:--:-- --:--:-- --:--:--  544k
/Users/rvimieiro/Documents/ufpe/aulas/data_science
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total  

#### Obtendo as informações das páginas

Agora vamos filtrar as informações desejadas e criar os diversos data frames para cada modelo.
Os exemplos serão criados com as páginas locais, mas podem ser facilmente adaptados para funcionarem online com a biblioteca/módulo requests.

Vamos seguir o mesmo procedimento para processar as páginas com os detalhes de cada modelo. Primeiro vamos inspecionar o código para identificar as seções que contêm as informações que queremos. Depois, implementamos o código para filtrar essas informações. Vamos usar a página com informações sobre o [iPhone 5S 16GB](../2017.1/data/scraping/buscape/smartphone-apple-iphone-5s-16gb.html) como exemplo.

Inspecionando o código vemos que a informação sobre os produtos está localizada na tag `li` com atributo class *offers-list__item* dentro da tag `section` com atributo `data-tab-content=precos`. Dentro deste item precisamos recuperar: o nome do vendedor, o preço à vista do aparelho, e o link para o site do vendedor. 

Visualmente o site não define o nome do vendedor explicitamente. Porém, se avaliarmos a imagem com a logo do vendedor, vemos que ela possui um texto alternativo (caso a imagem não carregue) que define o vendedor. Vamos usar essa informação como o nome do vendedor (na maioria dos itens avaliados esse texto correspondia de fato ao nome do vendedor, mas não podemos ter certeza de que esse será sempre o caso). As informações do vendedor estão armazenadas junto do link para seu site. Tal link possui o atributo class *store*. O texto do link é a logo do vendedor (tag `img`) que tem o atributo *alt* com o nome que usaremos para identificá-lo.

In [3]:
page = BeautifulSoup(open('../2017.1/data/scraping/buscape/smartphone-apple-iphone-5s-16gb',\
                          encoding='latin1'),'html.parser')
section = page.find('section',attrs={'data-tab-content':'precos'})
for item in section.find_all('li',class_='offers-list__item', log_id=re.compile('.*')):
    tmp = item.find('a',class_='store')
#    print(tmp)
    link = tmp.get('href')
    nome = tmp.find('img').get('alt')
    print(nome,link)
    

Wal-Mart http://www.buscape.com.br/redirect_prod?pagina=1&pos=1&pg=4&id=77&prod_id=168086187&emp_id=255256&nc=179180722520170202163226&az=fec1804a8a8a22785af604cd4b7889de&cn=1813727836
Mercado Livre http://www.buscape.com.br/redirect_prod?pagina=1&pos=2&pg=4&id=77&prod_id=224594656&emp_id=50248&nc=179180722520170202163226&az=40021f7c4322b02bd13819985be173c1&cn=1813727836
DouGShoP http://www.buscape.com.br/redirect_prod?pagina=1&pos=3&pg=4&id=77&prod_id=216725477&emp_id=1121457&nc=179180722520170202163226&az=eb2e049d51e4ce77d278d1a3cc09801d&cn=1813727836
KaBuM! http://www.buscape.com.br/redirect_prod?pagina=1&pos=4&pg=4&id=77&prod_id=214939756&emp_id=72420&nc=179180722520170202163226&az=17c911e52aaf27c5e5b9e5b2cbffdc8b&cn=1813727836
Carrefour http://www.buscape.com.br/redirect_prod?pagina=1&pos=5&pg=4&id=77&prod_id=219954123&emp_id=51010&nc=179180722520170202163226&az=1f40d952c72368379533b435db1022a5&cn=1813727836
Casas Bahia http://www.buscape.com.br/redirect_prod?pagina=1&pos=6&pg=4&i

**Obs.:** Se voltarmos à página no navegador, percebemos que os links acima se referem somente à primeira página. Vemos que, nesse exemplo específico, existem outros vendedores listados na página 2. Poderíamos usar `requests` para continuar a navegação e recuperar a informação dos outros vendedores, porém, vemos que o link se refere a um diretório não autorizado (o acesso ao diretório /ajax/ é proíbido no arquivo robots.txt). Portanto, vamos restringir nossa coleta somente à primeira página.

Continuando o processamento da página, agora que obtivemos o nome do vendedor e o link do produto, vamos coletar o preço à vista do aparelho. Novamente, voltamos à página e verificamos o elemento HTML que contém essa informação. A tag é `div` e ela possui o atributo `class` com a palavra *price*. O texto associado à tag contém o preço do aparelho.


Finalmente, após obter todas as informações desejadas, podemos construir um data frame com os dados.


In [15]:
page = BeautifulSoup(open('../2017.1/data/scraping/buscape/smartphone-apple-iphone-5s-16gb',\
                          encoding='latin1'),'html.parser')
section = page.find('section',attrs={'data-tab-content':'precos'})
aux = []
for item in section.find_all('li',class_='offers-list__item', log_id=re.compile('.*')):
    tmp = item.find('a',class_='store')
    link = tmp.get('href')
    nome = tmp.find('img').get('alt')
    preco = precoToFloat(item.find('div', class_='price').text)
    #print(nome,preco,link)
    #print(dict(zip(['vendedor','preco','link'],[nome,preco,link])))
    aux.append(dict(zip(['vendedor','preco','link'],[nome,preco,link])))
dfIPhone5S16GB = pd.DataFrame(aux,columns=['vendedor','preco','link'])
dfIPhone5S16GB

,vendedor,preco,link
0,Wal-Mart,1498.98,http://www.buscape.com.br/redirect_prod?pagina...
1,Mercado Livre,1599.00,http://www.buscape.com.br/redirect_prod?pagina...
2,DouGShoP,1381.25,http://www.buscape.com.br/redirect_prod?pagina...
3,KaBuM!,1529.90,http://www.buscape.com.br/redirect_prod?pagina...
4,Carrefour,1549.00,http://www.buscape.com.br/redirect_prod?pagina...
5,Casas Bahia,1495.12,http://www.buscape.com.br/redirect_prod?pagina...
6,PontoFrio.com,1580.07,http://www.buscape.com.br/redirect_prod?pagina...
7,Extra.com.br,1550.00,http://www.buscape.com.br/redirect_prod?pagina...
8,Magazine Luiza,1439.91,http://www.buscape.com.br/redirect_prod?pagina...
9,Americanas.com,1519.05,http://www.buscape.com.br/redirect_prod?pagina...


**Obs 2.:** Começamos a observar dados interessantes na tabela gerada. Vemos, por exemplo, que um mesmo vendedor aparece múltiplas vezes. Algumas vezes com preços idênticos, outras com preços distintos. Qual o significado disso? Será que houve algum erro na coleta e os dados foram repetidos? Será que há algum erro no site? Ao examinar novamente a página, percebemos que na verdade essa repetição se refere a múltiplas cores disponíveis. Algumas cores são mais caras que outras, logo a diferença no preço.

Agora temos de completar a informação na tabela com o modelo e a capacidade. Antes de continuar o processamento das outras páginas.

In [216]:
dfIPhone5S16GB['modelo'] = 'iPhone 5S'
dfIPhone5S16GB['capacidade'] = 16
dfIPhone5S16GB

,vendedor,preco,link,modelo,capacidade
0,Wal-Mart,1549.00,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
1,Mercado Livre,1699.00,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
2,DouGShoP,1381.25,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
3,KaBuM!,1529.90,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
4,Carrefour,1549.00,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
5,Casas Bahia,1495.12,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
6,PontoFrio.com,1580.07,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
7,Extra.com.br,1550.00,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
8,Magazine Luiza,1529.91,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16
9,Americanas.com,1614.05,http://www.buscape.com.br/redirect_prod?pagina...,iPhone 5S,16


Para automatizar o processo e evitar replicação de código, podemos criar uma função com o processamento acima e aplicá-la a cada modelo encontrado na página inicial de busca.

In [16]:
def coletaDadosModelo(modelo,capacidade,pagina):
    '''Coleta as informacoes de preco e o nome do vendedor na pagina referida'''
    page = BeautifulSoup(open(pagina, encoding='latin1'),'html.parser')
    section = page.find('section',attrs={'data-tab-content':'precos'})
    aux = [dict(zip(['modelo','capacidade','vendedor','preco','link'],\
                    [modelo,capacidade,item.find('a',class_='store').find('img').get('alt'),\
                                                 precoToFloat(item.find('div', class_='price').text),\
                                                 item.find('a',class_='store').get('href')]))\
           for item in section.find_all('li',class_='offers-list__item', log_id=re.compile('.*'))]
    return pd.DataFrame(aux,columns=['modelo','capacidade','vendedor','preco','link'])


coletaDadosModelo('iPhone 5S',16,'../2017.1/data/scraping/buscape/smartphone-apple-iphone-5s-16gb')

,modelo,capacidade,vendedor,preco,link
0,iPhone 5S,16,Wal-Mart,1498.98,http://www.buscape.com.br/redirect_prod?pagina...
1,iPhone 5S,16,Mercado Livre,1599.00,http://www.buscape.com.br/redirect_prod?pagina...
2,iPhone 5S,16,DouGShoP,1381.25,http://www.buscape.com.br/redirect_prod?pagina...
3,iPhone 5S,16,KaBuM!,1529.90,http://www.buscape.com.br/redirect_prod?pagina...
4,iPhone 5S,16,Carrefour,1549.00,http://www.buscape.com.br/redirect_prod?pagina...
5,iPhone 5S,16,Casas Bahia,1495.12,http://www.buscape.com.br/redirect_prod?pagina...
6,iPhone 5S,16,PontoFrio.com,1580.07,http://www.buscape.com.br/redirect_prod?pagina...
7,iPhone 5S,16,Extra.com.br,1550.00,http://www.buscape.com.br/redirect_prod?pagina...
8,iPhone 5S,16,Magazine Luiza,1439.91,http://www.buscape.com.br/redirect_prod?pagina...
9,iPhone 5S,16,Americanas.com,1519.05,http://www.buscape.com.br/redirect_prod?pagina...


#### Consolidando as informações em uma única tabela

Agora que definimos a função de coleta das informações detalhadas sobre os aparelho para cada vendedor, podemos aplicá-la nos links descobertos na página inicial da busca para obter uma tabela consolidada de todos os aparelhos. Note que a função opera sobre as páginas locais (download que fizemos anteriormente). Observe também que ela retorna um data frame para cada modelo/capacidade. Assim, devemos usar a função [concat](http://pandas.pydata.org/pandas-docs/stable/merging.html) de pandas para reunir todas as informações em um único data frame. Em outras palavras, devemos aplicar ```coletaDadosModelo``` em cada link e concatenar o resultado.

```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

Devemos observar, contudo, um pequeno detalhe. A função ```coletaDadosModelo``` espera como parâmetro o caminho local onde a página está armazenada, mas a variável `celulares` contém os links originais. Dessa forma, devemos substituir o prefixo do site pelo caminho local.

In [22]:
dadosCelulares= pd.concat(map(lambda item: coletaDadosModelo(item[0][0],\
                                             item[0][1],\
                                             re.sub('http.*/','../2017.1/data/scraping/buscape/',item[1])),\
                              celulares.items()), ignore_index=True)
print(dadosCelulares.shape)
dadosCelulares

(291, 5)


,modelo,capacidade,vendedor,preco,link
0,iPhone 7,32,Peixe Urbano,3149.90,http://www.buscape.com.br/redirect_prod?pagina...
1,iPhone 7,32,Wal-Mart,3499.00,http://www.buscape.com.br/redirect_prod?pagina...
2,iPhone 7,32,Mercado Livre,3699.90,http://www.buscape.com.br/redirect_prod?pagina...
3,iPhone 7,32,Fnac.com,3499.00,http://www.buscape.com.br/redirect_prod?pagina...
4,iPhone 7,32,DouGShoP,2931.65,http://www.buscape.com.br/redirect_prod?pagina...
5,iPhone 7,32,KaBuM!,3149.10,http://www.buscape.com.br/redirect_prod?pagina...
6,iPhone 7,32,Saraiva.com.br,3324.05,http://www.buscape.com.br/redirect_prod?pagina...
7,iPhone 7,32,Casas Bahia,3079.12,http://www.buscape.com.br/redirect_prod?pagina...
8,iPhone 7,32,Extra.com.br,3079.12,http://www.buscape.com.br/redirect_prod?pagina...
9,iPhone 7,32,PontoFrio.com,3079.12,http://www.buscape.com.br/redirect_prod?pagina...


#### Salvando as informações consolidadas em um arquivo

Para finalizar, podemos salvar as informações processadas em um arquivo [csv](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html) ou diretamente numa planilha do [Excel](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html).

In [244]:
dadosCelulares.to_csv('../2017.1/data/scraping/dadosConsolidadosIPhone7.csv', index=False)

dadosCelulares.to_excel('../2017.1/data/scraping/dadosConsolidadosIPhone7.xlsx', index=False)

## Exercícios

2. Escreva trechos de código para responder às seguintes perguntas:
  1. Qual o preço médio de cada modelo de iPhone (independente da capacidade ou cor)?
  2. Qual é a loja mais barata em média?
  3. Qual modelo possui a maior diferença de preço?
  4. Qual a variação de preço por modelo e capacidade?
1. Modifique a parte de coleta dos dados da página inicial da busca para também armazenar o número de avaliações feitas ao produto e a nota média exibida (número de estrelas)  